##### Load modules

In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

##### Inference on test data

In [2]:
df = pd.read_pickle('test.pickle')
df = df.sort_values('row_id')
df = df.drop(columns='row_id')

In [3]:
models = [lgb.Booster(model_file='{}.model'.format(i)) for i in range(4)]

In [4]:
def batch_inference(models, df, n_batch):
    
    result = []
    n_batch = int(n_batch)
    
    for idx in np.arange(0, df.shape[0], n_batch):
        progress = idx / df.shape[0] * 100
        progress = np.round(progress, 2)
        print('\r', progress, end='')
        
        start = idx
        end = (idx + n_batch)
        batch = df[start:end]
                
        pred = [model.predict(batch) for model in models]
        pred = np.mean(pred, axis=0)
        
        result.append(pred)
        
    result = np.concatenate(result)
    
    return result

In [5]:
y_pred = batch_inference(models, df, 1e4)
y_pred = np.clip(y_pred, 0, None)
y_pred = np.expm1(y_pred)

 99.98

In [7]:
df = pd.read_csv('sample_submission.csv')

In [8]:
df['meter_reading'] = y_pred

In [9]:
df.to_csv('submission.csv', index=False)